### 1. Document Preparation

In [ ]:
# Import library to work with excel
import pandas as pd

import os
folder = r"C:\Users\UserName\FolderRoute\FileName"
print(os.listdir(folder)) 

In [ ]:
# create dataframe with the indicators of interest
print(df_DataWorldIndicators.head())
df_DataWorldIndicators = pd.read_excel(r"C:\Users\UserName\FolderRoute\FileName.xlsx") 
df_DataWorldIndicators.columns

In [ ]:
# Reshape from wide → long

# Identify year columns
year_cols = [col for col in df_DataWorldIndicators.columns if "[YR" in col]

# Melt to long format
df_long = df_DataWorldIndicators.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)

# Extract numeric year
df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)

# Convert values to numeric (replace '..' with NaN)
df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce")

# Filter for 2000-2024
df_long = df_long[(df_long["year"] >= 2000) & (df_long["year"] <= 2024)]

# Preview
df_long.head()

In [ ]:
# Filter Indicators - Example using economic indicators

economic_indicators = [
    "GDP per capita (current US$)",
    "GNI per capita, Atlas method (current US$)",
    "GDP growth (annual %)",
    "Gini index",
    "Poverty headcount ratio at $4.20 a day (2021 PPP) (% of population)",
    "Population, total",
    "Central government debt, total (% of GDP)",
    "Current health expenditure (% of GDP)"
]

df_econ = df_long[df_long["Series Name"].isin(economic_indicators)]
df_econ.head

#### 2. How to generate world maps

In [ ]:
import plotly.express as px

# Filter data per indicator, using GDP per capita (current US$) as an example
df_plot = df_econ[df_econ["Series Name"] == "GDP per capita (current US$)"].copy()

# Ensure numeric columns
df_plot["value"] = pd.to_numeric(df_plot["value"], errors="coerce")
df_plot["year"] = pd.to_numeric(df_plot["year"], errors="coerce")

# Create the choropleth map
fig = px.choropleth(
    df_plot,
    locations="Country Code", 
    color="value",
    hover_name="Country Name",
    animation_frame="year",
    color_continuous_scale="RdPu",
    title="GDP per capita (current US$), 2000–2024"
)

# Save the interactive map as HTML
fig.write_html("GDP_per_capita_current_USD.html")
# Display the map in notebook
fig.show()

### 3. How to generate temporal charts (line chart)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Filter your data - using Life expectancy at birth, total (years) indicator as example
df_plot = df_health[df_health["Series Name"] == "Life expectancy at birth, total (years)"].copy()
df_plot["value"] = pd.to_numeric(df_plot["value"], errors="coerce")
df_plot["year"] = pd.to_numeric(df_plot["year"], errors="coerce")
df_plot = df_plot[df_plot["year"] != 2024]

# List of countries
countries = df_plot["Country Name"].unique()

# Create traces for each country
fig = go.Figure()

for country in countries:
    df_country = df_plot[df_plot["Country Name"] == country]
    fig.add_trace(
        go.Scatter(
            x=df_country["year"],
            y=df_country["value"],
            mode="lines+markers",
            name=country,
            visible=(country == countries[0])  # Only first country visible by default
        )
    )

# Create dropdown buttons
buttons = []

for i, country in enumerate(countries):
    # Set visibility: only selected country is True
    visibility = [False] * len(countries)
    visibility[i] = True

    buttons.append(
        dict(
            label=country,
            method="update",
            args=[{"visible": visibility},
                  {"title": f"Life Expectancy in {country}"}]
        )
    )

# Add dropdown to layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.15,  # position of dropdown
        y=0.8
    )],
    title=f"Life Expectancy in {countries[0]}",
    xaxis_title="Year",
    yaxis_title="Life Expectancy (years)",
    template="plotly_white"
)

# Show the figure
fig.show()

# Save as HTML
fig.write_html("Life_Expectancy_by_Country_Dropdown.html")

#### 4. How to generate barr graphs showing the ten highest and lowest countries

In [ ]:
# Using Life expectancy at birth, total (years) indicator as example

# Filter Life Expectancy
df_plot = df_health[df_health["Series Name"] == "Life expectancy at birth, total (years)"].copy()

# Ensure numeric
df_plot["value"] = pd.to_numeric(df_plot["value"], errors="coerce")
df_plot["year"] = pd.to_numeric(df_plot["year"], errors="coerce")

# Drop missing values
df_plot = df_plot.dropna(subset=["value", "Country Name"])

# Select specific year (e.g., 2023)
year_to_plot = 2023
df_latest = df_plot[df_plot["year"] == year_to_plot]

# Top 10 and Bottom 10
df_top10 = df_latest.nlargest(10, "value")
df_bottom10 = df_latest.nsmallest(10, "value")
df_combined = pd.concat([df_top10, df_bottom10]).reset_index(drop=True)

# Create horizontal bar chart
fig = px.bar(
    df_combined,
    x="value",
    y="Country Name",
    orientation="h",
    color="value",
    color_continuous_scale="Viridis",
    text="value",  # Show values on bars
    title=f"Top 10 and Bottom 10 Countries by Life Expectancy in {year_to_plot}",
    labels={"value": "Life Expectancy (years)", "Country Name": "Country"}
)

# Round bar values to 1 decimal
fig.update_traces(texttemplate='%{x:.1f}')

# Reverse y-axis so highest is on top
fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    template="plotly_white"
)

# Show chart
fig.show()

# Save as interactive HTML
fig.write_html(f"LifeExpectancy_TopBottom10_{year_to_plot}.html")

#### 5. Comparative analysis plots

In [ ]:
# Using UHC Service Coverage Index vs Life Expectancy es example

import plotly.express as px
import statsmodels.api as sm

# Create an interactive scatter plot
# Each point represents a country
fig = px.scatter(
    df_scatter,
    x="uhc_index",                    # UHC index on the x-axis
    y="life_expectancy",              # Life expectancy on the y-axis
    hover_name="Country Name",         # Show country name on hover
    title=f"UHC Service Coverage Index vs Life Expectancy ({year_to_plot})",
    labels={
        "uhc_index": "UHC Service Coverage Index",
        "life_expectancy": "Life Expectancy at Birth (years)"
    },
    trendline="ols"                    # Add a linear regression trendline
)

# Display graph in white background
fig.update_layout(template="plotly_white")

# Display the figure
fig.show()

# Save the interactive scatter plot as HTML
# Note: Avoid spaces and special characters in the file name
fig.write_html("UHC_vs_LifeExpectancy_2020.html")

#### 6. How to calculate the correlation coefficient: Pearson correlation

In [ ]:
# Calculate correlation between UHC index and life expectancy
correlation_matrix = df_scatter[["uhc_index", "life_expectancy"]].corr()

correlation_matrix

#### 7. Bubble plots with population size

In [ ]:
# Example used GDP per Capita and HCI

fig = px.scatter(
    df_compare_2020,
    x="gdp_per_capita",
    y="hci",
    size="population",           # Bubble size
    hover_name="Country Name",
    title="GDP per Capita vs Human Capital Index (2020) — Bubble Plot",
    labels={
        "gdp_per_capita": "GDP per Capita (current US$)",
        "hci": "Human Capital Index (0-1)",
        "population": "Population"
    },
    trendline="ols",
    color_discrete_sequence=["blue"],
    size_max=60,
)

fig.update_layout(template="plotly_white")
fig.show()
fig.write_html("GDP_vs_HCI_PopSize.html")